In [14]:
import matplotlib as plt
import pandas as pd
import networkx as nx
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch import Tensor
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx
from torch.utils.data import Dataset, DataLoader
from torch_geometric.nn import SAGEConv
from itertools import product

In [2]:
# Load data
signal_df = pd.read_csv('Dijet_bb_pt10_15_dw.csv')
background_df = pd.read_csv('Dijet_qq_pt10_15_dw.csv')

In [3]:
# Separate Jet 0 and Jet 1 data & combine signal/ background
sig_jet0 = signal_df[signal_df.columns[signal_df.columns.str.contains("Jet0")]]
back_jet0 = background_df[background_df.columns[background_df.columns.str.contains("Jet0")]]
train_df = pd.concat([s0, b0])

sig_jet1 = signal_df[signal_df.columns[signal_df.columns.str.contains("Jet1")]]
b_jet1 = background_df[background_df.columns[background_df.columns.str.contains("Jet1")]]
test_df = pd.concat([sig_jet0, back_jet0])

In [17]:
def no_edges_graph(df):
    G = nx.Graph()

    # Add all columns as nodes
    num_nodes = len(list(df.columns))
    nodes = list(df.columns)
    for node in nodes:
        G.add_node(node)
    
    # Add node features based on target_column
    for node in G.nodes(): 
        # Filter rows where target_column matches node
        node_features = df[df[node] == node].iloc[:, 1:].values  # Assuming the first column is an identifier
        if len(node_features) > 0:
            G.nodes[node]['x'] = torch.tensor(node_features[0], dtype=torch.float)
        else:
            G.nodes[node]['x'] = torch.tensor([0.0] * (df.shape[1] - 1), dtype=torch.float)
    
    # Convert to PyTorch Geometric Data object
    data = from_networkx(G)
    data.y = torch.ones(data.num_nodes, dtype=torch.long)
    data.train_mask = torch.ones(data.num_nodes, dtype=torch.bool)
    data.test_mask = torch.ones(data.num_nodes, dtype=torch.bool)
    combined_graph = Data(
        x=data.x,
        edge_index=data.edge_index,
        y=data.y,
        train_mask=data.train_mask,
        test_mask=data.test_mask
    )
    return data

In [18]:
train_graph = no_edges_graph(train_df)
test_graph = no_edges_graph(test_df)

In [19]:
train_graph

Data(x=[170, 169], edge_index=[2, 0], y=[170], train_mask=[170], test_mask=[170])

In [20]:
test_graph

Data(x=[170, 169], edge_index=[2, 0], y=[170], train_mask=[170], test_mask=[170])

In [21]:
# Define GNN model
class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()
        self.conv1 = SAGEConv(in_channels=test_graph.num_node_features, out_channels=16)
        self.conv2 = SAGEConv(in_channels=16, out_channels=2)
    
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

model = GNN()

In [24]:
# Training parameters
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
def train(model, data, optimizer, criterion):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

for epoch in range(10):
    loss = train(model, train_graph, optimizer, criterion)
    print(f'Epoch {epoch}, Loss: {loss}')

# Evaluation
def test(model, data):
    model.eval()
    _, pred = model(data).max(dim=1)
    correct = pred[data.test_mask].eq(data.y[data.test_mask]).sum().item()
    acc = correct / data.test_mask.sum().item()
    return acc

accuracy = test(model, test_graph)
print(f'Accuracy: {accuracy}')

Epoch 0, Loss: 0.006283882074058056
Epoch 1, Loss: 0.0054906983859837055
Epoch 2, Loss: 0.004792987834662199
Epoch 3, Loss: 0.004182164091616869
Epoch 4, Loss: 0.003649361664429307
Epoch 5, Loss: 0.0031861518509685993
Epoch 6, Loss: 0.0027846621815115213
Epoch 7, Loss: 0.0024373605847358704
Epoch 8, Loss: 0.0021374018397182226
Epoch 9, Loss: 0.0018787606386467814
Accuracy: 1.0
